# Import des modules

In [1]:
import string
from collections import Counter

import nltk
import pandas as pd
import numpy as np
import sklearn
import gensim
import spacy

# Plots
import matplotlib.pyplot as plt
%matplotlib inline

# Chargement des données
![Analyse de sentiments](https://modernmarketing.co.za/wp-content/uploads/2019/10/Developing-Customer-Centric-Strategy-Through-Automation-1-1.jpg)

In [2]:
# Dataset critiques IMDB
df_train = pd.read_csv("train.csv.gz")
df_valid = pd.read_csv("test.csv.gz")

# Explorez les données

In [3]:
df_train.head()

,text,score
0,I rented I AM CURIOUS-YELLOW from my video sto...,4
1,"""I Am Curious: Yellow"" is a risible and preten...",1
2,If only to avoid making this type of film in t...,1
3,This film was probably inspired by Godard's Ma...,2
4,"Oh, brother...after hearing about this ridicul...",1


In [4]:
df_valid.head()

,text,score
0,I love sci-fi and am willing to put up with a ...,3
1,"Worth the entertainment value of a rental, esp...",4
2,its a totally average film with a few semi-alr...,3
3,STAR RATING: ***** Saturday Night **** Friday ...,2
4,"First off let me say, If you haven't enjoyed a...",4


# Préparation du jeu de données

On a des critiques qui ont un score entre 1 et 10.

On va dire qu'une critique est positive si le score est plus grand que 5 et négative sinon.

Créez la colonne "positive" qui contient 0 pour les critiques négatives et 1 pour les positives

In [ ]:
# Si ça passe c'est ok
nb_positives = (df_train["positive"] == 1).sum()
assert nb_positives == 12500, f"nb_positives = {nb_positives} au lieu de 12500"

# Bag Of Words preprocessing

On récupère toutes les fonctions de preprocessing pour convertir les textes en matrice de features

In [ ]:
# nécessaire la 1ere fois
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

STOP_WORDS = set(stopwords.words('english'))

def extract_tokens(text):
    res = []
    for sent in nltk.sent_tokenize(text):
        res += nltk.word_tokenize(sent)
    return res

def clean_tokens(tokens):
    return [token.lower() for token in tokens if token not in string.punctuation]

def remove_stop_words(tokens):
    return [token for token in tokens if token not in STOP_WORDS]

def text2tokens(text):
    tokens = extract_tokens(text)
    tokens = clean_tokens(tokens)
    tokens = remove_stop_words(tokens)
    return tokens

scikit-learn a toutes les fonctionalités dont on a besoin : https://scikit-learn.org/stable/modules/classes.html?highlight=text#module-sklearn.feature_extraction.text

Utilisez ces nouveaux outils pour générer les matrices X_train et X_valid à partir de df_train["text"] et df_valid["text"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectoriser = ##

In [ ]:
X_train = ##

In [ ]:
X_valid = ##

Quelle est la taille du vocabulaire utilisé par votre vectorizer?

In [ ]:
# On prépare le vecteur à prédire
y_train = # 
y_valid = # 

# Un peu de Machine learning

On va maintenant utiliser scikit learn pour apprendre une régression logistique sur nos données d'entrainement : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

Etape 1 : Créez le modèle

Etape 2 : fit(X, y)

Etape 3 : Evaluons les performances

Commencez par instancier le modèle

Lancez l'apprentissage sur l'ensemble de train

### Affichez la matrice de confusion
https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

Explorez les options 'normalize' : Quelle information est la plus pertinente ?

## Tracez la courbe ROC
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_roc_curve.html?highlight=roc%20curve#sklearn.metrics.plot_roc_curve


## Calculer l'AUC : Area Under the ROC Curve
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html?highlight=auc#sklearn.metrics.auc

# Au tour de Word2Vec

Commencez par rajouter la colonne tokens aux dataframes df_train et df_valid.
Vous allez y stocker les tokens des textes en utilisant les fonctions données en début du notebook

In [ ]:
# Si ça passe c'est ok
assert "tokens" in df_train.columns, df_train.columns
assert "tokens" in df_valid.columns, df_valid.columns

On va utiliser gensim et sa fonction Word2Vec : https://radimrehurek.com/gensim/models/word2vec.html

Crééz un modèle appelé w2v :
 - Choisissez des vecteur de taille 300,
 - entrainez le modèle sur les tokens d'entrainement (df_train)
 - pensez à utiliser tous les cpus à votre disposition

Est-ce que king - man + woman nous donne queen ? Oui/Non et surtout pourquoi?

indice : utilisez la méthode most_similar du modèle

Explorez un peu le modèle, est-ce que vous trouvez des choses intéressantes?

La méthode doesnt_match peut être intéressante

# Machine learning avec word2vec

Grace au modèle word2vec on est capable de transformer un mot en vecteur et ces vecteurs ont des propriétés intéressantes.

Mais on travaille avec des textes composés de plusieurs mots et de plusieurs phrases. Comment utiliser le modèle sur un texte complet.

La possibilité la plus souvent utilisée est de prendre la moyenne des vecteurs associés aux mots du texte. Voyons voir ce que ça donne comme performance vs le TF-IDF du début du TP.

Créez la fonction text2vec qui à un texte retourne le vecteur qui correspond à la moyenne des vecteurs associés aux mots du texte. (regardez la méthode mean des array numpy)

Créez la matrice d'entrainement X_train_w2v à partir de df_train et de text2vec

Créez la matrice de validation X_valid_w2v à partir de df_valid et de text2vec

On entraine une régression linéaire

On plot la matrice de confusion

On plot la courbe ROC

On calcule l'AUC

Alors Word2Vec ça résoud tout les problèmes?

# Et si on charge le word2vec de google

Il est dispo sur miriel045 : /tmp/jacq/GoogleNews-vectors-negative300.bin.gz
Copiez le sur le /tmp de votre machine, pas dans votre home! (1.5G)

Vous pouvez aussi utiliser [spacy](https://spacy.io/usage/models) en chargeant les modèles : en_core_web_md ou en_core_web_lg

Est-ce que le modèle obtenu avec ces vecteurs est plus performant que le notre? Que le TF-IDF?